**Libraries**

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import datetime
from datetime import date, timedelta
from sklearn.cluster import KMeans
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import plotly.offline as py
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm
from keras.models import Sequential
from keras.layers import LSTM,Dense
from keras.layers import Dropout
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

Using TensorFlow backend.


In [2]:
hospital  = pd.read_csv("../input/hospitalbedloc/HospitalBedsIndiaLocations.csv")

 **Hospitals in India**

In [3]:
hospital.head()

,Sno,State/UT,NumPrimaryHealthCenters_HMIS,NumCommunityHealthCenters_HMIS,NumSubDistrictHospitals_HMIS,NumDistrictHospitals_HMIS,TotalPublicHealthFacilities_HMIS,NumPublicBeds_HMIS,NumRuralHospitals_NHP18,NumRuralBeds_NHP18,NumUrbanHospitals_NHP18,NumUrbanBeds_NHP18,Latitude,Longitude
0,1,Andaman & Nicobar Islands,27,4,NaN,3,34,1246,27,575,3,500,11.6234,92.7265
1,2,Andhra Pradesh,1417,198,31.0,20,1666,60799,193,6480,65,16658,15.9129,79.7400
2,3,Arunachal Pradesh,122,62,NaN,15,199,2320,208,2136,10,268,27.0844,93.6053
3,4,Assam,1007,166,14.0,33,1220,19115,1176,10944,50,6198,26.1433,91.7898
4,5,Bihar,2007,63,33.0,43,2146,17796,930,6083,103,5936,25.5941,85.1376


**This file contain Health Management Information System data and National Health Profile Data and we deal diffrently with them**

In [4]:
hmis = hospital.loc[:,['Sno','State/UT','Latitude','Longitude','NumPrimaryHealthCenters_HMIS','NumCommunityHealthCenters_HMIS','NumSubDistrictHospitals_HMIS','NumDistrictHospitals_HMIS','TotalPublicHealthFacilities_HMIS','NumPublicBeds_HMIS']]

In [5]:
nhp = hospital.loc[:,['Sno','State/UT','Latitude','Longitude','NumRuralHospitals_NHP18','NumRuralBeds_NHP18','NumUrbanHospitals_NHP18','NumUrbanBeds_NHP18']]

In [6]:
hmis = hmis.rename(columns={"State/UT":"State","NumPrimaryHealthCenters_HMIS":"Primary","NumCommunityHealthCenters_HMIS":"Community","NumSubDistrictHospitals_HMIS":"SubDistrict","NumDistrictHospitals_HMIS":"District","TotalPublicHealthFacilities_HMIS":"Total","NumPublicBeds_HMIS":"Public beds"})

In [7]:
nhp = nhp.rename(columns={'State/UT':"State",'NumRuralHospitals_NHP18':"Rural_hospitals",'NumRuralBeds_NHP18':"Rural_beds",'NumUrbanHospitals_NHP18':"Urban_hospitals",'NumUrbanBeds_NHP18':"Urban_beds"})

In [8]:
hmis = hmis.fillna(0)

In [9]:
hmis.head()

,Sno,State,Latitude,Longitude,Primary,Community,SubDistrict,District,Total,Public beds
0,1,Andaman & Nicobar Islands,11.6234,92.7265,27,4,0.0,3,34,1246
1,2,Andhra Pradesh,15.9129,79.7400,1417,198,31.0,20,1666,60799
2,3,Arunachal Pradesh,27.0844,93.6053,122,62,0.0,15,199,2320
3,4,Assam,26.1433,91.7898,1007,166,14.0,33,1220,19115
4,5,Bihar,25.5941,85.1376,2007,63,33.0,43,2146,17796


**HMIS Hospitals in World Map**

In [10]:
import folium
india = folium.Map(location=[20.5937,78.9629 ], zoom_start=5,tiles='cartodbpositron')

for lat, lon,State,Primary ,Community, SubDistrict, District, Total in zip(hmis['Latitude'], hmis['Longitude'],hmis['State'],hmis['Total'],hmis['Primary'],hmis['Community'],hmis['SubDistrict'],hmis['District']):
    folium.CircleMarker([lat, lon],
                        radius=5,
                        color='red',
                      popup =('State:' + str(State) + '<br>'
                             'Total Hospitals:' + str(Total) + '<br>'
                              'Primary :' + str(Primary) + '<br>'
                             'Community:' + str(Community) + '<br>'
                             'Sub District:'+ str(SubDistrict) + '<br>'
                             'District:'+ str(District) + '<br>'

                             ),
                        
                        fill_color='red',
                        fill_opacity=0.7 ).add_to(india)
india

In [11]:
hmis = hmis.sort_values(by='Total', ascending=False)

In [12]:
fig = px.bar(hmis[['State', 'Total']].sort_values('Total', ascending=False), 
             y="Total", x="State", color='State', 
             log_y=True, template='ggplot2', title='Total Hospitals')
fig.show()

In [13]:
fig = px.bar(hmis[['State', 'Primary']].sort_values('Primary', ascending=False), 
             y="Primary", x="State", color='State', 
             log_y=True, template='ggplot2', title='Primary Hospitals')
fig.show()

In [14]:
fig = px.bar(hmis[['State', 'Community']].sort_values('Community', ascending=False), 
             y="Community", x="State", color='State', 
             log_y=True, template='ggplot2', title='Community Hospitals')
fig.show()

In [15]:
fig = px.bar(hmis[['State', 'District']].sort_values('District', ascending=False), 
             y="District", x="State", color='State', 
             log_y=True, template='ggplot2', title='District Hospitals')
fig.show()

In [16]:
fig = px.bar(hmis[['State', 'SubDistrict']].sort_values('SubDistrict', ascending=False), 
             y="SubDistrict", x="State", color='State', 
             log_y=True, template='ggplot2', title='SubDistrict Hospitals')
fig.show()

In [17]:
nhp = nhp.fillna(0)
nhp['Total_hospitals'] = nhp['Rural_hospitals'] + nhp['Urban_hospitals']
nhp['Total_beds'] = nhp['Rural_beds'] + nhp['Urban_beds']


In [18]:
nhp.head()

,Sno,State,Latitude,Longitude,Rural_hospitals,Rural_beds,Urban_hospitals,Urban_beds,Total_hospitals,Total_beds
0,1,Andaman & Nicobar Islands,11.6234,92.7265,27,575,3,500,30,1075
1,2,Andhra Pradesh,15.9129,79.7400,193,6480,65,16658,258,23138
2,3,Arunachal Pradesh,27.0844,93.6053,208,2136,10,268,218,2404
3,4,Assam,26.1433,91.7898,1176,10944,50,6198,1226,17142
4,5,Bihar,25.5941,85.1376,930,6083,103,5936,1033,12019


# Total Beds

In [19]:
ms = nhp.sort_values(by=['Total_beds'],ascending=False)
ms = ms.head(30)
fig = px.funnel(ms, x='Total_beds', y='State')
fig.show()

In [20]:
fig = px.bar(nhp[['State', 'Total_hospitals']].sort_values('Total_hospitals', ascending=False), 
             y="Total_hospitals", x="State", color='State', 
             log_y=True, template='ggplot2', title='Total Hospitals')
fig.show()

In [21]:
fig = px.bar(nhp[['State', 'Rural_hospitals']].sort_values('Rural_hospitals', ascending=False), 
             y="Rural_hospitals", x="State", color='State', 
             log_y=True, template='ggplot2', title='Rural Hospitals')
fig.show()

In [22]:
fig = px.bar(nhp[['State', 'Rural_beds']].sort_values('Rural_beds', ascending=False), 
             y="Rural_beds", x="State", color='State', 
             log_y=True, template='ggplot2', title='Rural Beds')
fig.show()

In [23]:
fig = px.bar(nhp[['State', 'Urban_hospitals']].sort_values('Urban_hospitals', ascending=False), 
             y="Urban_hospitals", x="State", color='State', 
             log_y=True, template='ggplot2', title='Urban Hospitals')
fig.show()

In [24]:
fig = px.bar(nhp[['State', 'Urban_beds']].sort_values('Urban_beds', ascending=False), 
             y="Urban_beds", x="State", color='State', 
             log_y=True, template='ggplot2', title='Urban beds')
fig.show()